In [48]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional, Flatten, Activation, Embedding, Concatenate, Input, Dense, Dropout, MaxPool2D
from keras.layers import Reshape, Flatten, Conv1D, MaxPool1D, Embedding,BatchNormalization, LSTM,merge, Conv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, Model
import keras.utils as ku 
import numpy as np
from keras import regularizers
import pickle as pkl
tokenizer = Tokenizer()
from html import unescape
from nltk.stem import SnowballStemmer
import re
from nltk.corpus import stopwords
import sys
sys.stdout.write('hello')

hello

In [2]:
#stop=set(stopwords.words('english'))
#stem=SnowballStemmer('english')
df=pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')#, sep='\t')
df_un=pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')#, sep='\t')
df['bin'] = df.target>=0.5

In [3]:
def re_clean(text):
    #c.append('')
    #if len(c)%1000==0:
        #pass
        #print(len(c))
    text=text.lower()
    symbol = """!#$%^&*();:\t\\\"!\{\}\[\]<>-\?\-\\\"—\.,1234567890"""
    text=re.sub("\'ll", ' will', text)
    text=re.sub("\'ve", ' have', text)
    text=re.sub("\'s", ' is', text)
    text=re.sub('[{}]'.format(symbol),' ', text)
    #text=re.sub('[\W]',' ', text)
    text=re.sub('\n',' ', text)
    text=re.sub(' +',' ', text)
    text=re.sub('^\s', '', text)
    #text=' '.join([stem.stem(i) for i in text.split() if not i in stop])
    #text=[i for i in text.split() if i not in stop]
    return text #' '.join(text)

In [4]:
sys.stdout.write('Cleaning start')
df['text']=df.comment_text.apply(re_clean)
df_un['text']=df_un.comment_text.apply(re_clean)
sys.stdout.write('Cleaning done')

Cleaning start
Cleaning done


In [5]:
!cd /kaggle/input/glove840b300dtxt && ls

glove.840B.300d.txt


In [6]:
%%time
tokenizer.fit_on_texts(list(df.text)+list(df_un.text))
sys.stdout.write('tokenizing done!')

tokenizing done!
CPU times: user 1min 52s, sys: 568 ms, total: 1min 53s
Wall time: 1min 53s


In [7]:
m=220

In [8]:
%%time
X_train_seq_trunc = tokenizer.texts_to_sequences(df.text)

X_train_seq_trunc = pad_sequences(X_train_seq_trunc, maxlen=m, truncating='post', padding='post')
#from sklearn.model_selection import train_test_split
y_train = df.bin
#X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X_train_seq_trunc, y_train, test_size=0.1, random_state=37)

#assert X_valid_emb.shape[0] == y_valid_emb.shape[0]
#assert X_train_emb.shape[0] == y_train_emb.shape[0]

#print('Shape of validation set:',X_valid_emb.shape)
#del X_train_seq_trunc
del df

sys.stdout.write('sequencing done!')

sequencing done!
CPU times: user 1min 46s, sys: 1.4 s, total: 1min 47s
Wall time: 1min 47s


In [9]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'and': 3,
 'is': 4,
 'of': 5,
 'a': 6,
 'in': 7,
 'that': 8,
 'it': 9,
 'you': 10,
 'for': 11,
 'i': 12,
 'are': 13,
 'not': 14,
 'have': 15,
 'be': 16,
 'on': 17,
 'this': 18,
 'they': 19,
 'with': 20,
 'as': 21,
 'was': 22,
 'or': 23,
 'but': 24,
 'he': 25,
 'will': 26,
 'we': 27,
 'if': 28,
 'all': 29,
 'what': 30,
 'their': 31,
 'by': 32,
 'so': 33,
 'who': 34,
 'from': 35,
 'your': 36,
 'at': 37,
 'no': 38,
 'do': 39,
 'people': 40,
 'would': 41,
 'about': 42,
 'has': 43,
 'there': 44,
 'an': 45,
 'more': 46,
 'his': 47,
 'one': 48,
 'just': 49,
 'like': 50,
 'can': 51,
 'trump': 52,
 'out': 53,
 'when': 54,
 'up': 55,
 'our': 56,
 "don't": 57,
 'how': 58,
 'them': 59,
 'my': 60,
 'get': 61,
 'should': 62,
 'than': 63,
 'us': 64,
 'been': 65,
 'were': 66,
 'only': 67,
 'time': 68,
 'any': 69,
 'some': 70,
 'other': 71,
 'because': 72,
 'now': 73,
 'think': 74,
 'those': 75,
 'why': 76,
 'many': 77,
 'know': 78,
 'good': 79,
 'then': 80,
 'which': 81,
 'even':

In [18]:
import gc
gc.collect()

0

In [39]:
%%time
NB_WORDS=len(tokenizer.word_index)
GLOVE_DIM=300


emb_matrix = np.zeros((NB_WORDS+1, GLOVE_DIM))
def emb():
    with open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt') as f:
        for n, i in enumerate(f):
            i = i.split()
            if tokenizer.word_index.get(' '.join(i[:-300])):
                #print(' '.join(i[:-300]))
                emb_matrix[tokenizer.word_index.get(' '.join(i[:-300]))] = np.array(i[-300:])
            if n%500000==0:
                sys.stdout.write(n)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 4.54 ms


In [40]:
emb()

0
500000
1000000
1500000
2000000


In [41]:
emb_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.27204 , -0.06203 , -0.1884  , ...,  0.13015 , -0.18317 ,
         0.1323  ],
       [ 0.31924 ,  0.06316 , -0.27858 , ...,  0.082745,  0.097801,
         0.25045 ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.71899 , -0.6913  ,  0.042406, ..., -0.71715 , -0.015302,
         0.20028 ]])

In [42]:
num_filters = 256
filter_sizes = [3,4,5]
dropout = 0.5
embedding_dim = 300
vocabulary_size = NB_WORDS
sequence_length = m

In [43]:
inp1=Input((m,))
emb=Embedding(NB_WORDS+1,embedding_dim, weights=[emb_matrix],trainable=False)(inp1)
#lstm1=Bidirectional(LSTM(128, return_sequences=True))(emb)
#drop1=Dropout(dropout)(lstm1)
#lstm2=Bidirectional(LSTM(128, return_sequences=True))(drop1)
#drop2=Dropout(dropout)(lstm2)
#lstm2=Bidirectional(LSTM(128, return_sequences=False))(drop2)
#dense_lstm=Dense(128, activation='relu')(lstm2)
reshape = Reshape((sequence_length,embedding_dim,1))(emb)
conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)
concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])#, maxpool_3])
flatten = Flatten()(concatenated_tensor)
#dense_con=Dense(128, activation='relu')(flatten)
#conc=conc=merge.multiply(([dense_con, dense_lstm]))
batch1=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', 
                              gamma_initializer='ones', moving_mean_initializer='zeros', 
                              moving_variance_initializer='ones', beta_regularizer=None,
                              gamma_regularizer=None, beta_constraint=None,
                              gamma_constraint=None)(flatten)
dense3=Dense(128, activation='relu')(batch1)
drop4=Dropout(0.5)(dense3)
dense4=Dense(128, activation='relu')(drop4)
"""batch2=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', 
                          gamma_initializer='ones', moving_mean_initializer='zeros', 
                          moving_variance_initializer='ones', beta_regularizer=None,
                          gamma_regularizer=None, beta_constraint=None,
                          gamma_constraint=None)(dense4)"""
#conc=Concatenate()([dense7, out1])
drop5=Dropout(0.5)(dense4)
dense7=Dense(128, activation='relu')(drop5)
out=Dense(1, activation='sigmoid')(dense7)
model= Model(inp1, out)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [45]:
from keras.optimizers import Adam
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

checkpoint = ModelCheckpoint('jigsaw.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
model.compile(loss='binary_crossentropy', optimizer=adam,
              metrics=['accuracy'], )

gc.collect()

0

In [47]:
sys.stdout.write('Training...')
history=model.fit(X_train_seq_trunc
                       , y_train
                       , epochs=8
                       , batch_size=2048
                       , validation_split= 0.1 #(X_valid_emb, y_valid_emb)
                       , verbose=1, callbacks=[checkpoint])
sys.stdout.write("Trained!")

Train on 1624386 samples, validate on 180488 samples
Epoch 1/8
 570768/1624386 [=========>....................] - ETA: 3:01 - loss: 0.2322 - acc: 0.9214

KeyboardInterrupt: 

In [ ]:
X_test_seq_trunc = tokenizer.texts_to_sequences(df_un.text)
X_test_seq_trunc = pad_sequences(X_test_seq_trunc, maxlen=m, truncating='post', padding='post')
gc.collect()

In [ ]:
from keras.models import load_model
model=load_model('jigsaw.hdf5')
pre=model.predict(X_test_seq_trunc, verbose=1)
df3=pd.DataFrame(pre, index=df_un.id, columns=['prediction'])
df3.to_csv('submission.csv')
sys.stdout.write('Done!')